In [1]:
%load_ext rpy2.ipython

In [2]:
import pandas as pd
import scipy.io as io
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from matplotlib import gridspec
import seaborn as sns
import datetime
from datetime import datetime, timedelta
import pickle
import itertools
import glob

In [3]:
import matplotlib.dates as dates
from scipy import stats
from scipy.stats import variation
import math
from sklearn import preprocessing
from scipy.signal import butter, filtfilt
import itertools

In [4]:
import matplotlib.pyplot as plt
#font = {'family': 'serif', 'serif': ['Palatino']}
font = {'family': 'serif', 'serif': ['Helvetica']}
plt.rc('font', **font)
plt.rc('text', usetex=False)

In [5]:
label_fontsize = 16
title_fontsize = 16

In [6]:
#Set up parameters for figure display
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8, 8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'axes.labelweight': 'bold',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

pylab.rcParams.update(params)
pylab.rcParams["font.family"] = "serif"
pylab.rcParams["font.weight"] = "heavy"
#pylab.rcParams.update({'figure.autolayout': True})

In [7]:
sns.set(font_scale=1.25, style="whitegrid")

In [8]:
%%R
#install.packages("BayesFactor", repos='http://cran.us.r-project.org')

NULL


In [9]:
%%R
rm(list = ls())  #remove elements from current workspace
library("plyr")
library("emmeans")
library("reshape2")
library("multcomp")

R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: survival

R[write to console]: Loading required package: TH.data

R[write to console]: Loading required package: MASS

R[write to console]: 
Attaching package: ‘TH.data’


R[write to console]: The following object is masked from ‘package:MASS’:

    geyser




In [10]:
%%R
basepath = "/rds/project/tb419/rds-tb419-bekinschtein/Sri/SpatialAttention_Drowsiness/"
behfilepath = "decAlertnessDecisionmaking_JNeuroscience2021/beh_data/csv_files/"

In [11]:
%%R
#read the csv file first (created by process_create_StimCodeRTs)
csvfile = paste(basepath,behfilepath,"responsecoding_stimcode_state.csv",sep="")
myoveralldata = read.csv(csvfile) # read csv file

In [12]:
%%R
myoveralldata <- subset(myoveralldata, select = -c(angle, difficulty, response, stimcode, accuracy, ovtrlnums))
myoveralldata <-subset(myoveralldata, state!='Relaxed')
head(myoveralldata)

  subj_idx       rt  stim state
1        2 0.904771 Right Alert
2        2 1.032423  Left Alert
3        2 1.145844  Left Alert
4        2 1.099559  Left Alert
5        2 1.228592  Left Alert
6        2 1.541085 Right Alert


In [13]:
%%R
min(myoveralldata$rt)

[1] 0.024684


In [14]:
%%R
unique(myoveralldata$state)

[1] Alert  Drowsy
Levels: Alert Drowsy Relaxed


In [15]:
%%R
aggstats_df <- do.call(data.frame, aggregate(. ~ subj_idx+stim+state, data = myoveralldata, 
                                  FUN = function(x) c(mn = mean(x), n = length(x) ) ) )
aggstats_df <- rename(aggstats_df,c("rt.mn" = "rt", "rt.n" = "trls"))

In [16]:
%%R
print(head(aggstats_df))
print(tail(aggstats_df))

  subj_idx stim state        rt trls
1        2 Left Alert 0.9980229   60
2        4 Left Alert 0.7128657   60
3        5 Left Alert 0.6360013   60
4        6 Left Alert 1.1625786   60
5        7 Left Alert 0.7787995   60
6        9 Left Alert 0.7738686   60
    subj_idx  stim  state        rt trls
123       41 Right Drowsy 1.2389740  147
124       42 Right Drowsy 1.0444405  298
125       43 Right Drowsy 0.6753177  193
126       44 Right Drowsy 1.2325250   65
127       45 Right Drowsy 1.9843120  211
128       46 Right Drowsy 1.7260631   39


In [17]:
%%R
packageVersion("lme4")

[1] ‘1.1.21’


In [18]:
%%R
# Step 3: Multilevel model with missing values ~ Hierarchical model
# First compare and choose the best model..

library(lmerTest)

model.null = lmer(rt ~ 1 + (1|subj_idx), data=aggstats_df,
                  REML=FALSE)

model.state = lmer(rt ~ state + (1|subj_idx), data=aggstats_df,
                   REML=FALSE)

model.Stim = lmer(rt ~ stim + (1|subj_idx), data=aggstats_df,
                  REML=FALSE)

model.StateStim = lmer(rt ~ state*stim + (1|subj_idx), data=aggstats_df,
                       REML=FALSE)

R[write to console]: Loading required package: lme4

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step




In [19]:
%%R
#compare the null model, with the state model..
print(anova(model.null,model.state))

Data: aggstats_df
Models:
model.null: rt ~ 1 + (1 | subj_idx)
model.state: rt ~ state + (1 | subj_idx)
            Df     AIC     BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
model.null   3 165.916 174.472 -79.958  159.916                             
model.state  4  74.017  85.426 -33.009   66.017 93.898      1  < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [20]:
%%R
#compare the null model, with the stim model..
print(anova(model.null,model.Stim))

Data: aggstats_df
Models:
model.null: rt ~ 1 + (1 | subj_idx)
model.Stim: rt ~ stim + (1 | subj_idx)
           Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)
model.null  3 165.92 174.47 -79.958   159.92                         
model.Stim  4 167.58 178.99 -79.789   159.58 0.3375      1     0.5613


In [21]:
%%R
#compare the null model, with the statestim model..
print(anova(model.null,model.StateStim))

Data: aggstats_df
Models:
model.null: rt ~ 1 + (1 | subj_idx)
model.StateStim: rt ~ state * stim + (1 | subj_idx)
                Df     AIC     BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)
model.null       3 165.916 174.472 -79.958  159.916                         
model.StateStim  6  76.132  93.245 -32.066   64.132 95.783      3  < 2.2e-16
                   
model.null         
model.StateStim ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [22]:
%%R
#the above analysis show that the statestimlevel model has the highest loglikelihood and the next
#highest is the statestim model, now we can make a direct comparison of them both..
print(anova(model.Stim,model.StateStim))

Data: aggstats_df
Models:
model.Stim: rt ~ stim + (1 | subj_idx)
model.StateStim: rt ~ state * stim + (1 | subj_idx)
                Df     AIC     BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)
model.Stim       4 167.578 178.986 -79.789  159.578                         
model.StateStim  6  76.132  93.245 -32.066   64.132 95.446      2  < 2.2e-16
                   
model.Stim         
model.StateStim ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [23]:
%%R
#the above analysis show that the statestim model is the most likely model..
#Step4 :let's analyse the same..
print(anova(model.StateStim))

Type III Analysis of Variance Table with Satterthwaite's method
           Sum Sq Mean Sq NumDF DenDF  F value Pr(>F)    
state      9.1412  9.1412     1    96 162.4633 <2e-16 ***
stim       0.0514  0.0514     1    96   0.9138 0.3415    
state:stim 0.0557  0.0557     1    96   0.9899 0.3223    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [24]:
%%R
#the above table shows that there is a main effect of state(alert, drowsy)

NULL
